In [6]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(Metrics)
library(rsq)

options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)


load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_num_finale.Rdata")

df_num <- df_num_finale

# Feature selection

In [7]:
library(leaps)

Best_Subset <-
    regsubsets(degenzaPostTraOSP~.,
               data =df_num,
               nbest = 1,      # 1 best model for each number of predictors
               nvmax = NULL,    # NULL for no limit on number of variables
               force.in = NULL, force.out = NULL,
               method = "exhaustive")
summary_best_subset <- summary(Best_Subset)

which.max(summary_best_subset$adjr2)

[1] 15

In [8]:
tmp <- summary_best_subset$which[18,]

tmp

(Intercept)                      age            degenzaPreTra 
                    TRUE                    FALSE                     TRUE 
           ratioPaO2FiO2         platCountTranspl                 albumina 
                    TRUE                     TRUE                     TRUE 
         surgeryDuration IntraoperativTransfected             plasmaFresco 
                    TRUE                    FALSE                     TRUE 
             plasmaPools           doseFibrinogen         CRIOprecipitated 
                    TRUE                     TRUE                     TRUE 
        CristalloidiComp           Albumina50Comp   noradrenalineEndSurger 
                    TRUE                     TRUE                     TRUE 
        intermedLactates             coldIschemia            organDonorAge 
                    TRUE                    FALSE                     TRUE 
        organDonorHeight organDonorDrawalSodiemia        organDonorGammaGT 
                    TRUE                     TRUE                     TRUE 
              realMELDNa 
                    TRUE

In [12]:
df_num_select <- subset(df_num, select= c(degenzaPreTra,ratioPaO2FiO2,platCountTranspl,albumina,surgeryDuration,plasmaFresco,plasmaPools,doseFibrinogen,CRIOprecipitated,CristalloidiComp,Albumina50Comp,noradrenalineEndSurger,intermedLactates,organDonorAge,organDonorHeight,organDonorDrawalSodiemia,organDonorGammaGT,realMELDNa,degenzaPostTraOSP))

In [13]:
dim(df_num_select)

[1] 1403   19

# Data partition

In [14]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df_num$degenzaPostTraOSP, SplitRatio = 0.7)
train  <- subset(df_num, sample == TRUE)
test   <- subset(df_num, sample == FALSE)


In [15]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample_select <- sample.split(df_num_select$degenzaPostTraOSP, SplitRatio = 0.7)
train_select  <- subset(df_num_select, sample_select == TRUE)
test_select   <- subset(df_num_select, sample_select == FALSE)


# Model

In [16]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

In [17]:
lm <- train(degenzaPostTraOSP ~ ., data = train,
               method = "lm",
               trControl = ctrl)

+ Fold01.Rep1: intercept=TRUE 
- Fold01.Rep1: intercept=TRUE 
+ Fold02.Rep1: intercept=TRUE 
- Fold02.Rep1: intercept=TRUE 
+ Fold03.Rep1: intercept=TRUE 
- Fold03.Rep1: intercept=TRUE 
+ Fold04.Rep1: intercept=TRUE 
- Fold04.Rep1: intercept=TRUE 
+ Fold05.Rep1: intercept=TRUE 
- Fold05.Rep1: intercept=TRUE 
+ Fold06.Rep1: intercept=TRUE 
- Fold06.Rep1: intercept=TRUE 
+ Fold07.Rep1: intercept=TRUE 
- Fold07.Rep1: intercept=TRUE 
+ Fold08.Rep1: intercept=TRUE 
- Fold08.Rep1: intercept=TRUE 
+ Fold09.Rep1: intercept=TRUE 
- Fold09.Rep1: intercept=TRUE 
+ Fold10.Rep1: intercept=TRUE 
- Fold10.Rep1: intercept=TRUE 
Aggregating results
Fitting final model on full training set


In [18]:
lm_select <- train(degenzaPostTraOSP ~ ., data = train_select,
               method = "lm",
               trControl = ctrl)

+ Fold01.Rep1: intercept=TRUE 
- Fold01.Rep1: intercept=TRUE 
+ Fold02.Rep1: intercept=TRUE 
- Fold02.Rep1: intercept=TRUE 
+ Fold03.Rep1: intercept=TRUE 
- Fold03.Rep1: intercept=TRUE 
+ Fold04.Rep1: intercept=TRUE 
- Fold04.Rep1: intercept=TRUE 
+ Fold05.Rep1: intercept=TRUE 
- Fold05.Rep1: intercept=TRUE 
+ Fold06.Rep1: intercept=TRUE 
- Fold06.Rep1: intercept=TRUE 
+ Fold07.Rep1: intercept=TRUE 
- Fold07.Rep1: intercept=TRUE 
+ Fold08.Rep1: intercept=TRUE 
- Fold08.Rep1: intercept=TRUE 
+ Fold09.Rep1: intercept=TRUE 
- Fold09.Rep1: intercept=TRUE 
+ Fold10.Rep1: intercept=TRUE 
- Fold10.Rep1: intercept=TRUE 
Aggregating results
Fitting final model on full training set


In [19]:
lm

Linear Regression 

985 samples
 21 predictor

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 887, 886, 888, 885, 887, 886, ... 
Resampling results:

  RMSE       Rsquared   MAE      
  0.6301635  0.2156569  0.4632772

Tuning parameter 'intercept' was held constant at a value of TRUE

In [20]:
lm_select

Linear Regression 

985 samples
 18 predictor

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 886, 887, 887, 885, 887, 887, ... 
Resampling results:

  RMSE       Rsquared   MAE      
  0.6261779  0.2150405  0.4587742

Tuning parameter 'intercept' was held constant at a value of TRUE

In [21]:
# collect resamples
results <- resamples(list(lm=lm,  lm_select=lm_select))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: lm, lm_select 
Number of resamples: 10 

MAE 
               Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
lm        0.4060027 0.4434169 0.4792149 0.4632772 0.4813364 0.4952001    0
lm_select 0.4078436 0.4302856 0.4522487 0.4587742 0.4776538 0.5296290    0

RMSE 
               Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
lm        0.5467081 0.6052129 0.6362953 0.6301635 0.6683488 0.6968645    0
lm_select 0.5763634 0.5889772 0.6219067 0.6261779 0.6459926 0.7097267    0

Rsquared 
               Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
lm        0.1388549 0.1521919 0.1850629 0.2156569 0.2654522 0.3440128    0
lm_select 0.1344978 0.1583088 0.2111127 0.2150405 0.2402664 0.3492660    0


In [22]:
vimp <- varImp(lm_select)
vimp
plot(vimp, cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.4)))

lm variable importance

                         Overall
realMELDNa               100.000
surgeryDuration           96.482
intermedLactates          92.813
noradrenalineEndSurger    72.621
ratioPaO2FiO2             59.487
organDonorDrawalSodiemia  46.872
degenzaPreTra             40.535
Albumina50Comp            38.662
plasmaPools               34.539
organDonorHeight          32.777
platCountTranspl          28.821
organDonorGammaGT         21.487
doseFibrinogen            19.649
CRIOprecipitated          17.463
CristalloidiComp          15.984
organDonorAge             12.302
plasmaFresco               4.512
albumina                   0.000

In [23]:
test_select$predicted <- predict(lm_select, newdata = test_select)
cor(test_select$degenzaPostTraOSP ,test_select$predicted)^2  #rsquared

[1] 0.2184078

In [24]:
mae(test_select$degenzaPostTraOSP ,test_select$predicted)

[1] 0.4379968

In [25]:
rmse(test_select$degenzaPostTraOSP, test_select$predicted)

[1] 0.5740004

In [26]:
#predicted vs actual

ggplot(data = test_select, aes(x = predicted, y = degenzaPostTraOSP)) +
  geom_point(alpha = 0.2, color = "blue") +
  geom_smooth(aes(x = predicted,
                  y = degenzaPostTraOSP), color="blue") +
  geom_line(aes(x = degenzaPostTraOSP,
                y = degenzaPostTraOSP), color = "black", 
            linetype = 2, size = 1)

`geom_smooth()` using method = 'loess' and formula 'y ~ x'



In [27]:
# residuals

ggplot(data = test_select, aes(x = predicted,
                      y = predicted - degenzaPostTraOSP)) +
  geom_point(alpha = 0.2, color = "blue") +
  geom_smooth(aes(x = predicted,
                  y = predicted - degenzaPostTraOSP),
              color="black")

`geom_smooth()` using method = 'loess' and formula 'y ~ x'

